In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.chains import RetrievalQA
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("../files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


In [16]:
retriever = vectorstore.as_retriever()

In [18]:
template = """다음 정보를 사용하여 질문에 답변하세요:

{context}

질문: {question}
이전 대화:
{chat_history}

답변:"""

prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=template
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

C:\Users\rich\AppData\Local\Temp\ipykernel_34352\3423423525.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [19]:
chain = StuffDocumentsChain(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    prompt=prompt,
)



KeyError: 'llm_chain'